In [1]:
import numpy as np
import tensorflow as tf

# Constant

## string

In [2]:
with tf.Session() as sess:
    text = tf.constant('Hello, TensorFlow!')

    print(sess.run(text))
    print("text is", text)

b'Hello, TensorFlow!'
text is Tensor("Const:0", shape=(), dtype=string)


text isn't native string, it is Tensor object

# formula

In [3]:
with tf.Session() as sess:
    c1 = tf.constant(1)
    c2 = tf.constant(2)
    formula = c1 + c2
    print("c1 + c2 is", sess.run(formula))

c1 + c2 is 3


## placeholder

In [4]:
with tf.Session() as sess:
    x = tf.placeholder(tf.int32)
    y = tf.placeholder(tf.int32)
    formula = x + y
    print("c1 + c2 is", sess.run(formula, { x: [1,2,3], y: [4,5,6] }))

c1 + c2 is [5 7 9]


# Train Linear Model

In [5]:
# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]
# training loop
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init) # reset values to wrong
    for i in range(1000):
      sess.run(train, {x:x_train, y:y_train})

    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# Find the paramater of Celsius to fahrenheit

In [6]:
# c = celsius = x
# f = fahrenheit = y

c = tf.placeholder(tf.float32, name="celsius")
f = tf.placeholder(tf.float32, name="fahrenheit")

b = tf.Variable(0, dtype=tf.float32, name="b")
m = tf.Variable(0, dtype=tf.float32, name="m")

# convert celsius to fahrenheit
linear_formula = m * c + b

# Error and loss
error = linear_formula - f
loss = tf.reduce_sum(tf.square(error))

# training optimizer
optimizer = tf.train.AdagradOptimizer(1)
train = optimizer.minimize(loss)

# training data
c_train = [0,5,12,20, 28]
f_train = [32,41,53.6,68, 82.4]

# training loop
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    max_loop, loop_count, loss_value, threshold = [10000, 0, 1, 1e-6] 
    
    while loop_count < max_loop and loss_value > threshold:
        loop_count = loop_count + 1
        loss_value = sess.run([loss, train], {c:c_train, f:f_train})[0]

    # print training result
    print("Loop count: %s, loss: %s"%(loop_count, loss_value))
    
    # use session.run to get the last values
    print("F = C * {0:.2n} + {1:.2n}".format(*sess.run([m, b])))
    
    tf.train.Saver().save(sess, "/tmp/model.ckpt")

Loop count: 4771, loss: 9.94973e-07
F = C * 1.8 + 32


In [7]:
with tf.Session() as sess:
    tf.train.Saver().restore(sess, "/tmp/model.ckpt")
    print(sess.run(linear_formula, {c:[38]}))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
[ 100.40061951]
